
# PEFT / QLoRA **(Colab · Python 3 · GPU T4)** — Llama 3.x Instruct · v2

Notebook actualizado para **Colab con CUDA 12.6**: incluye correcciones de instalación para **bitsandbytes** (rueda con soporte CUDA actual) y **Triton**, y mantiene ajustes de memoria/precisión para **T4 (16 GB)**.

**Objetivo**: adaptar un modelo **Llama 3.x Instruct** a un **ChatGPT especializado en Arquitectura de Software** mediante **PEFT (LoRA/IA3/AdaLoRA)** y **QLoRA**.

> Marcadores pedagógicos: **[TRANSFORMER]** indica dónde se usa la arquitectura Transformer. **[DATA TRANSFORM]** indica operaciones de transformación de datos.


In [ ]:
# ============================================================
# 0) Instalación robusta para Colab (CUDA 12.6 / T4) — EJECUTA PRIMERO
# ============================================================
# Limpieza de paquetes opcionales que suelen causar conflictos y bnb previo
!pip uninstall -y flash-attn xformers bitsandbytes || true

# Pila base fijada (estable) para evitar regresiones en Colab
!pip install -U "transformers==4.45.2" "accelerate==0.34.2" #   "datasets==2.20.0" "peft==0.13.2" "trl==0.11.4" "sentencepiece==0.2.0"

# bitsandbytes con binarios recientes (incluye CUDA 12.x)
!pip install -U --pre bitsandbytes

# Triton requerido por kernels/integraciones (alineado con PyTorch 2.5.x en Colab)
!pip install "triton>=3.0.0"

# (Opcional) Si quieres volver a instalar xformers:
!pip install xformers
# (Opcional) flash-attn suele ser innecesario en T4, pero si insistes:
!pip install flash-attn --no-build-isolation

In [ ]:

# ============================================================
# 1) Verificación de entorno + bitsandbytes (CUDA 12.6)
# ============================================================
import torch, platform, sys, os, glob
print("Python:", platform.python_version())
print("Torch:", torch.__version__, "| CUDA:", torch.version.cuda, "| CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

try:
    import bitsandbytes as bnb
    print("bitsandbytes:", getattr(bnb, "__version__", "unknown"))
    bnblibs = glob.glob(os.path.join(os.path.dirname(bnb.__file__), "libbitsandbytes_cuda*.so"))
    print("BNB libs:", bnblibs)
    if not bnblibs:
        print("⚠️  No se encontraron binarios CUDA de bitsandbytes. Considera reiniciar runtime y re-ejecutar la celda 0.")
except Exception as e:
    print("bitsandbytes import error:", e)


In [ ]:
# ============================================================
# 2) Configuración global (optimizada para T4 · 16 GB)
# ============================================================
import os
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

from dataclasses import dataclass

@dataclass
class Config:
    # Ruta raíz para guardar/cargar archivos en Google Drive
    DRIVE_ROOT = "/content/drive/MyDrive"

    # Modelo base a usar de Hugging Face Hub
    # Valores posibles: Cualquier modelo compatible con AutoModelForCausalLM (ej: "meta-llama/Llama-3-8B-Instruct", "mistralai/Mistral-7B-Instruct-v0.2")
    BASE_MODEL: str = "meta-llama/Llama-3.1-8B-Instruct"

    # Ruta local al archivo JSONL con el dataset de chat
    DATASET_LOCAL_JSONL: str = f"{DRIVE_ROOT}/datasets/arqsoft_chat.jsonl"

    # ID del dataset en Hugging Face Hub (si se usa en lugar de local)
    # Ejemplo: de ajibawa-2023/Software-Architecture, https://huggingface.co/datasets/ajibawa-2023/Software-Architecture
    DATASET_HF_ID: str | None = None

    # Directorio de salida para guardar checkpoints (punto de control del modelo) y el adaptador entrenado
    OUTPUT_DIR: str = f"{DRIVE_ROOT}/outputs/llama3_arqsoft_peft"

    # Nombre del adaptador PEFT (usado para guardar/cargar)
    ADAPTER_NAME: str = "arqsoft-qlora"

    # Número máximo de pasos de entrenamiento (iteraciones del optimizador)
    # Valores posibles: Un entero positivo. -1 para entrenar por número de épocas.
    MAX_STEPS: int = 10

    # Número de épocas completas sobre el dataset de entrenamiento, el MAX_STEPS>1, esta variable no se toma en cuenta.
    NUM_EPOCHS: int = 1

    # Tasa de aprendizaje para el optimizador
    # Valores posibles: Un flotante positivo pequeño (ej: 1e-5 a 5e-4)
    LEARNING_RATE: float = 2e-4

    # Tamaño del batch por dispositivo (GPU)
    # Valores posibles: Un entero positivo (usualmente 1 para QLoRA en T4 para ahorrar memoria)
    PER_DEVICE_BATCH_SIZE: int = 1

    # Número de pasos de acumulación de gradiente
    # Valores posibles: Un entero positivo. Batch efectivo = PER_DEVICE_BATCH_SIZE * GRADIENT_ACCUMULATION
    GRADIENT_ACCUMULATION: int = 16

    # Longitud máxima de secuencia para tokenizar (padding/truncation)
    # Valores posibles: Un entero positivo. Depende del modelo y dataset (ej: 512, 1024, 2048)
    MAX_SEQ_LEN: int = 1024

    # Proporción de pasos usados para calentamiento (warmup) del learning rate
    WARMUP_RATIO: float = 0.03

    # Frecuencia para registrar métricas de entrenamiento (en pasos)
    LOGGING_STEPS: int = 10

    # Frecuencia para evaluar el modelo en el dataset de validación (en pasos)
    # Valores posibles: Un entero positivo. Ignorado si eval_strategy="no".
    EVAL_STEPS: int = 100

    # Frecuencia para guardar checkpoints del modelo (en pasos)
    SAVE_STEPS: int = 200

    # Usar precisión bfloat16 (requiere GPU Ampere+ y soporte)
    # Valores posibles: bool (True/False). False para T4.
    USE_BF16: bool = False

    # Usar precisión float16 (más amplio soporte en GPUs como T4)
    # Valores posibles: bool (True/False). True para T4.
    USE_FP16: bool = True

    # Tipo de dato para la computación en 4 bits (bitsandbytes)
    # Valores posibles: "float16", "bfloat16" (si la GPU lo soporta)
    BNB_4BIT_COMPUTE_DTYPE: str = "float16"

    # Cargar el modelo en 4 bits usando bitsandbytes
    LOAD_IN_4BIT: bool = True

    # Tipo de cuantización de 4 bits (bitsandbytes)
    # Valores posibles: "nf4", "fp4"
    BNB_4BIT_QUANT_TYPE: str = "nf4"

    # Habilitar gradient checkpointing para ahorrar memoria
    # Valores posibles: bool (True/False). Recomendado en T4.
    GRADIENT_CHECKPOINTING: bool = True

    # Parámetro 'r' para LoRA: dimensión de los adaptadores
    # Valores posibles: Un entero positivo (ej: 8, 16, 32, 64)
    LORA_R: int = 16

    # Parámetro 'lora_alpha' para LoRA: factor de escalado
    # Valores posibles: Un entero positivo (ej: 16, 32, 64). Usualmente >= LORA_R.
    LORA_ALPHA: int = 32

    # Parámetro 'lora_dropout' para LoRA: dropout en los adaptadores
    # Valores posibles: Un flotante entre 0.0 y 1.0
    LORA_DROPOUT: float = 0.05

    # Módulos del modelo base a los que se aplicará LoRA
    TARGET_MODULES: tuple[str, ...] = ("q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj")

    # Tipo de tarea para PEFT (Causal Language Modeling para generación de texto)
    # Valores posibles: "CAUSAL_LM", "SEQ_CLS", etc.
    TASK_TYPE: str = "CAUSAL_LM"

    # Número máximo de tokens a generar durante la inferencia de prueba
    MAX_NEW_TOKENS: int = 256

    # Temperatura para el muestreo durante la generación (controla la aleatoriedad)
    TEMPERATURE: float = 0.2

    # Parámetro Top-P para el muestreo durante la generación (controla la diversidad)
    TOP_P: float = 0.95

CFG = Config()
CFG

In [ ]:
# ============================================================
# 3) Montar Google Drive para usar y almacenar datos
# ============================================================
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

# Ruta raíz del Google Drive montado
DRIVE_ROOT = "/content/drive/MyDrive"
# Carpeta para la caché de Hugging Face (modelos Llama 3.x, datasets (en caso se active su ID en Conf))
HF_ROOT = f"{DRIVE_ROOT}/hf_cache"
# Carpeta para almacenar datasets locales
DATA_ROOT = f"{DRIVE_ROOT}/datasets"
# Carpeta para guardar outputs del entrenamiento (checkpoints, adaptadores)
OUT_ROOT = f"{DRIVE_ROOT}/llama3_arqsoft_peft"

import os
os.makedirs(HF_ROOT, exist_ok=True)
os.makedirs(DATA_ROOT, exist_ok=True)
os.makedirs(OUT_ROOT, exist_ok=True)

# Redirige caches de Hugging Face (modelos/datasets) a Drive
os.environ["HF_HOME"] = HF_ROOT        # raíz HF (recomendado)
os.environ["HF_HUB_CACHE"] = f"{HF_ROOT}/hub"   # opcional fino

# Ajusta tu Config del notebook:
CFG.DATASET_LOCAL_JSONL = f"{DATA_ROOT}/arqsoft_chat.jsonl"
CFG.OUTPUT_DIR = OUT_ROOT

# --- Agregar logs aquí ---
print("\n--- Rutas y configuraciones de Google Drive ---")
print(f"DRIVE_ROOT (Raíz de Drive): {DRIVE_ROOT}")
print(f"HF_ROOT (Caché de Hugging Face): {HF_ROOT}")
print(f"DATA_ROOT (Datasets locales): {DATA_ROOT}")
print(f"OUT_ROOT (Outputs de entrenamiento): {OUT_ROOT}")
print(f"CFG.DATASET_LOCAL_JSONL (Ruta del dataset en Config): {CFG.DATASET_LOCAL_JSONL}")
print(f"CFG.OUTPUT_DIR (Directorio de outputs en Config): {CFG.OUTPUT_DIR}")
print("---------------------------------------------")
# --------------------------

In [ ]:
import os
print(f"Ruta del dataset configurada: {CFG.DATASET_LOCAL_JSONL}")
if os.path.exists(CFG.DATASET_LOCAL_JSONL):
    print(f"Archivo encontrado. Tamaño: {os.path.getsize(CFG.DATASET_LOCAL_JSONL)} bytes")
else:
    print("Archivo NO encontrado en la ruta configurada.")

In [ ]:

# ============================================================
# 4) Login a Hugging Face (necesario para descargar Llama 3.x)
# ============================================================
# 4.1) Genera tu User Access Token en https://huggingface.co/settings/tokens (scope: "read" para descargar; "write" si vas a subir)
# 4.2) En Colab: usa input seguro
from getpass import getpass
from huggingface_hub import login

token = getpass("HF token (no se mostrará): ")
login(token=token)  # almacena el token en ~/.cache/huggingface

from huggingface_hub import whoami
print(whoami())



In [ ]:
# ============================================================
# 5) Carga Tokenizer y Modelo 4-bit (QLoRA)
#    [TRANSFORMER] Aquí se instancia el Transformer Llama 3.x
# ============================================================
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import time

print("--- Iniciando carga de Tokenizer y Modelo 4-bit (QLoRA) ---")
start_time = time.time()

# Paso 1: Configurar BitsAndBytesConfig
# Esta configuración especifica cómo se cargará y usará el modelo en 4 bits.
print("Paso 1: Configurando BitsAndBytesConfig...")
# Input principal: Variables de configuración (LOAD_IN_4BIT, BNB_4BIT_QUANT_TYPE, BNB_4BIT_COMPUTE_DTYPE)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=CFG.LOAD_IN_4BIT,
    bnb_4bit_quant_type=CFG.BNB_4BIT_QUANT_TYPE,
    bnb_4bit_compute_dtype=getattr(torch, CFG.BNB_4BIT_COMPUTE_DTYPE),
)
print("BitsAndBytesConfig configurado.")
# Output principal: Un objeto BitsAndBytesConfig
print(f"  BitsAndBytesConfig details: {bnb_config}")


# Paso 2: Cargar el Tokenizer
# AutoTokenizer.from_pretrained carga el tokenizer asociado al modelo base especificado.
# Es responsable de convertir texto en IDs numéricos (tokens) y viceversa.
print(f"Paso 2: Cargando Tokenizer desde {CFG.BASE_MODEL}...")
tokenizer_start_time = time.time()
# Input principal: CFG.BASE_MODEL (string con el nombre/ruta del modelo)
tokenizer = AutoTokenizer.from_pretrained(
    CFG.BASE_MODEL,
    use_fast=True, # Usar la versión rápida del tokenizer (si está disponible)
    padding_side="right" # Configurar dónde añadir el padding (importante para modelos causales)
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token # Asegurar que haya un token de padding
tokenizer_end_time = time.time()
print(f"Tokenizer cargado en {tokenizer_end_time - tokenizer_start_time:.2f} segundos.")
print(f"Pad token configurado: {tokenizer.pad_token}, Pad token ID: {tokenizer.pad_token_id}")
# Output principal: Un objeto Tokenizer
print(f"  Tokenizer class: {type(tokenizer)}")
print(f"  Tokenizer vocab size: {tokenizer.vocab_size}")


# Paso 3: Cargar el Modelo en 4-bit
# AutoModelForCausalLM.from_pretrained carga el modelo base pre-entrenado.
# La cuantización BitsAndBytesConfig se aplica durante este proceso si LOAD_IN_4BIT es True.
print(f"Paso 3: Cargando Modelo 4-bit desde {CFG.BASE_MODEL}...de Hugging Face")
model_start_time = time.time()
# Input principal: CFG.BASE_MODEL (string), quantization_config (objeto BitsAndBytesConfig), device_map
model = AutoModelForCausalLM.from_pretrained(
    CFG.BASE_MODEL,
    quantization_config=bnb_config, # Aplicar la configuración de cuantización
    device_map="auto", # Distribuir las capas del modelo automáticamente entre los dispositivos disponibles (GPU)
    trust_remote_code=False # No ejecutar código arbitrario del Hub (generalmente True por seguridad)
)
model_end_time = time.time()
print(f"Modelo 4-bit cargado en {model_end_time - model_start_time:.2f} segundos.")
# Output principal: Un objeto AutoModelForCausalLM (cuantizado si se configuró así)
print(f"  Model class: {type(model)}")
print(f"  Model device: {model.device}") # Note: device_map="auto" might show base device or a range


# Configuraciones adicionales del modelo cargado
model.config.use_cache = False # Deshabilitar caché durante el entrenamiento (ahorra memoria, útil para gradient checkpointing)
model.config.pad_token_id = tokenizer.pad_token_id # Asegurar que el modelo conozca el ID del token de padding

end_time = time.time()
print(f"--- Proceso de carga completado en {end_time - start_time:.2f} segundos ---")

In [ ]:
# ============================================================
# 6) Preparación PEFT (LoRA sobre QLoRA)
#    [TRANSFORMER] Inyectamos adaptadores LoRA en q/k/v/o y MLP
# ============================================================
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import time # Import time for logging

print("--- Iniciando preparación PEFT ---")
start_time = time.time()

# Paso 1: Preparar el modelo base cargado en 4-bit para entrenamiento k-bit (QLoRA)
# Esta función aplica pre-procesamiento al modelo, como habilitar gradient checkpointing
# si está configurado, y preparar los embeddings para el entrenamiento en 4-bit.
print("Paso 1: Preparando modelo para entrenamiento k-bit...")
# Input principal: El objeto 'model' cargado en 4-bit de la celda anterior.
print(f"  Input a prepare_model_for_kbit_training: Model of type {type(model)}")
model = prepare_model_for_kbit_training(model)
print("Modelo preparado para entrenamiento k-bit.")
# Output principal: El mismo objeto 'model', pero con hooks y configuraciones para k-bit training.


# Paso 2: Definir la configuración de LoRA
# LoraConfig define los hiperparámetros del adaptador LoRA que se inyectará.
# Estos hiperparámetros controlan el tamaño (r, lora_alpha) y la regularización (lora_dropout)
# de las matrices de bajo rango, así como a qué módulos del modelo base se aplicarán (target_modules).
print("Paso 2: Definiendo LoraConfig con los siguientes parámetros:")
print(f"  r: {CFG.LORA_R}")
print(f"  lora_alpha: {CFG.LORA_ALPHA}")
print(f"  lora_dropout: {CFG.LORA_DROPOUT}")
print(f"  target_modules: {CFG.TARGET_MODULES}")
print(f"  task_type: {CFG.TASK_TYPE}")
peft_config = LoraConfig(
    r=CFG.LORA_R,
    lora_alpha=CFG.LORA_ALPHA,
    lora_dropout=CFG.LORA_DROPOUT,
    target_modules=list(CFG.TARGET_MODULES), # Aseguramos que sea una lista si target_modules es tupla
    task_type=CFG.TASK_TYPE,
    bias="none" # Generalmente "none" para fine-tuning de modelos generativos
)
print("LoraConfig definido.")
# Output principal: Un objeto LoraConfig.
print(f"  Output de LoraConfig: {peft_config}")


# Paso 3: Obtener el modelo PEFT (inyectar los adaptadores LoRA)
# Esta función toma el modelo base y la configuración LoRA, y devuelve
# un "PeftModel", que es el modelo base con los adaptadores LoRA inyectados
# y configurado para entrenar solo esos adaptadores.
print("Paso 3: Inyectando adaptadores LoRA con get_peft_model...")
# Inputs: El modelo preparado para k-bit training y el objeto LoraConfig.
print(f"  Input 1 a get_peft_model: Model of type {type(model)}")
print(f"  Input 2 a get_peft_model: LoraConfig of type {type(peft_config)}")
model = get_peft_model(model, peft_config)
print("Adaptadores LoRA inyectados. Modelo PEFT creado.")
# Output principal: Un objeto PeftModel.
print(f"  Output de get_peft_model: Model of type {type(model)}")


# Mostrar parámetros entrenables
# Esto imprime cuántos parámetros totales tiene el modelo base y cuántos
# parámetros adicionales (los de LoRA) serán entrenados.
print("\n--- Resumen de parámetros entrenables ---")
model.print_trainable_parameters()
print("----------------------------------------")

end_time = time.time()
print(f"--- Preparación PEFT completada en {end_time - start_time:.2f} segundos ---")


> **Alternativas PEFT**: IA3/AdaLoRA (cambia `peft_config`).

In [ ]:
# ============================================================
# 7) Carga de Dataset (formato chat)
# ============================================================
from datasets import load_dataset, Dataset
import json, os
import time # Import time for logging

print("--- Iniciando carga de Dataset ---")
start_time = time.time()

# Define una función para cargar el dataset desde un archivo local JSONL o desde Hugging Face Hub
# Prioriza el archivo local si se especifica y existe.
def load_chat_dataset(local_path: str | None, hf_id: str | None):
    # --- Logs de entrada a la función ---
    # Muestra los valores de los inputs local_path y hf_id que recibe la función.
    print(f"  Intentando cargar dataset desde: local_path='{local_path}', hf_id='{hf_id}'")
    # -----------------------------------
    if local_path and os.path.exists(local_path):
        print(f"  Archivo local encontrado en: {local_path}. Cargando desde JSONL...")
        with open(local_path, "r", encoding="utf-8") as f:
            # Lee cada línea como un objeto JSON (asumiendo que cada línea es un ejemplo de chat)
            records = [json.loads(line) for line in f]
        # Convierte la lista de diccionarios en un objeto Dataset de Hugging Face
        dataset = Dataset.from_list(records)
        print(f"  Dataset cargado desde JSONL local. Número de ejemplos: {len(dataset)}")
        return dataset
    elif hf_id:
        print(f"  Archivo local no encontrado o no especificado. Intentando cargar desde Hugging Face Hub: {hf_id}")
        # Carga el dataset directamente desde Hugging Face Hub
        # Assume split 'train' by default, you might need to adjust this based on the dataset structure
        dataset = load_dataset(hf_id, split="train")
        print(f"  Dataset cargado desde Hugging Face Hub. Número de ejemplos: {len(dataset)}")
        return dataset
    else:
        print("  No se especificó archivo local ni ID de Hugging Face. Usando dataset mini de ejemplo.")
        # Si no se especifica ninguna fuente válida, crea un pequeño dataset de ejemplo
        mini = [
            {"messages": [
                {"role":"system","content":"Eres un asistente experto en Arquitectura de Software."},
                {"role":"user","content":"Compara API Gateway vs Service Mesh con pros/cons y cuándo usar cada uno."},
                {"role":"assistant","content":"API Gateway gestiona tráfico norte-sur, auth, rate-limit; Mesh cubre este-oeste con mTLS, retries, observabilidad. Usa Gateway en el borde y Mesh intra-servicios cuando la malla sea compleja."}
            ]},
            {"messages": [
                {"role":"system","content":"Eres un asistente experto en Arquitectura de Software."},
                {"role":"user","content":"Diseña un patrón EDA en Kafka para fidelización al 99.99%."},
                {"role":"assistant","content":"Particiones y RF≥3, acks=all, min.insync.replicas=2, DLQ, Schema Registry, idempotent producer, outbox, SLO/SLI y alertas por latencia/lag."}
            ]},
        ]
        dataset = Dataset.from_list(mini)
        print(f"  Dataset mini de ejemplo creado. Número de ejemplos: {len(dataset)}")
        return dataset

# Llama a la función para cargar el dataset utilizando las rutas/IDs de la configuración
# Inputs a la función: CFG.DATASET_LOCAL_JSONL y CFG.DATASET_HF_ID definidos en la celda 2.
raw_ds = load_chat_dataset(CFG.DATASET_LOCAL_JSONL, CFG.DATASET_HF_ID)

# --- Log del primer ejemplo del dataset cargado ---
# Muestra la estructura y contenido del primer ejemplo del dataset cargado (output de la función).
print("\n--- Primer ejemplo del Dataset cargado (Output de load_chat_dataset) ---")
print(raw_ds[0])
print("---------------------------------------")
# --------------------------------------------

end_time = time.time()
print(f"--- Proceso de carga de Dataset completado en {end_time - start_time:.2f} segundos ---")

In [ ]:
# ============================================================
# 8) Transformación de datos
#    [DATA TRANSFORM] chat_template → tokenización → labels (pad→-100)
# ============================================================

def format_and_tokenize(example):
    text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False
    )
    tokenized = tokenizer(
        text,
        truncation=True,
        max_length=CFG.MAX_SEQ_LEN,
        padding="max_length",
        return_tensors=None,
    )

    pad_id = tokenizer.pad_token_id
    input_ids = tokenized["input_ids"]
    if input_ids and isinstance(input_ids[0], list):
        labels = [
            [tok if tok != pad_id else -100 for tok in seq]
            for seq in input_ids
        ]
    else:
        labels = [tok if tok != pad_id else -100 for tok in input_ids]


    tokenized["labels"] = labels
    return tokenized

raw_ds.map(format_and_tokenize, remove_columns=raw_ds.column_names)

# Divide el dataset procesado en conjuntos de entrenamiento y validación
split = raw_ds.map(format_and_tokenize, remove_columns=raw_ds.column_names).train_test_split(test_size=0.20, seed=42)
train_ds, val_ds = split["train"], split["test"]

# Muestra el número de ejemplos en cada conjunto
print(f"Dataset de Entrenamiento: {len(train_ds)} ejemplos")
print(f"Dataset de Validación: {len(val_ds)} ejemplos")

In [ ]:
# ============================================================
# 9) Entrenamiento (TRL SFTTrainer)
# ============================================================
try:
    from trl import SFTTrainer, SFTConfig
except ModuleNotFoundError:
    import subprocess
    import sys
    print("Instalando TRL (trl==0.11.4)...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "trl==0.11.4"])
    from trl import SFTTrainer, SFTConfig

from transformers import default_data_collator

# Determine eval_strategy based on max_steps
# Si MAX_STEPS > 0, la estrategia de evaluación se establece en "no" para evitar
# evaluaciones intermedias basadas en pasos que podrían no alinearse bien con los checkpoints.
# Si MAX_STEPS <= 0 (entrenamiento por épocas), la evaluación se basa en pasos ("steps").
eval_strategy = "steps" if CFG.MAX_STEPS <= 0 else "no"

# SFTConfig define los hiperparámetros y configuraciones para el entrenamiento de fine-tuning supervisado (SFT).
# Es similar a TrainingArguments de la librería transformers, pero adaptada para SFT en TRL.
print("--- Configurando SFTConfig para el entrenamiento ---")
# Inputs principales: Variables de configuración definidas en la clase Config (celda 2).
print(f"  Output directory: {CFG.OUTPUT_DIR}")
print(f"  Max sequence length: {CFG.MAX_SEQ_LEN}")
print(f"  Per device train batch size: {CFG.PER_DEVICE_BATCH_SIZE}")
print(f"  Gradient accumulation steps: {CFG.GRADIENT_ACCUMULATION}")
print(f"  Learning rate: {CFG.LEARNING_RATE}")
print(f"  Logging steps: {CFG.LOGGING_STEPS}")
print(f"  Eval strategy: {eval_strategy}")
print(f"  Eval steps: {CFG.EVAL_STEPS}")
print(f"  Save steps: {CFG.SAVE_STEPS}")
print(f"  Use BF16: {CFG.USE_BF16}")
print(f"  Use FP16: {CFG.USE_FP16}")
print(f"  Warmup ratio: {CFG.WARMUP_RATIO}")
print(f"  Max steps (calculated): {CFG.MAX_STEPS if CFG.MAX_STEPS > 0 else -1}")
print(f"  Num train epochs (calculated): {CFG.NUM_EPOCHS if CFG.MAX_STEPS <= 0 else 1000}")
print(f"  Gradient checkpointing: {CFG.GRADIENT_CHECKPOINTING}")

sft_config = SFTConfig(
    output_dir=CFG.OUTPUT_DIR,
    max_seq_length=CFG.MAX_SEQ_LEN,
    per_device_train_batch_size=CFG.PER_DEVICE_BATCH_SIZE,
    gradient_accumulation_steps=CFG.GRADIENT_ACCUMULATION,
    learning_rate=CFG.LEARNING_RATE,
    logging_steps=CFG.LOGGING_STEPS,
    eval_strategy=eval_strategy, # Use the determined strategy
    eval_steps=CFG.EVAL_STEPS,
    save_steps=CFG.SAVE_STEPS,
    bf16=CFG.USE_BF16,
    fp16=CFG.USE_FP16,
    warmup_ratio=CFG.WARMUP_RATIO,
    max_steps=CFG.MAX_STEPS if CFG.MAX_STEPS > 0 else -1, # Use -1 for no max steps
    num_train_epochs=CFG.NUM_EPOCHS if CFG.MAX_STEPS <= 0 else 1000, # Set epochs to a large value if max_steps is used
    gradient_checkpointing=CFG.GRADIENT_CHECKPOINTING,
    report_to=["none"], # Deshabilita reportes a plataformas como Weights & Biases por defecto
)
print("SFTConfig configurado.")
# Output principal: Un objeto SFTConfig.


# SFTTrainer es la clase principal de TRL para realizar el fine-tuning supervisado.
# Abstrae el bucle de entrenamiento estándar, la preparación de datos, la optimización,
# y la evaluación, especialmente optimizado para modelos grandes y PEFT.
print("\n--- Inicializando SFTTrainer ---")
# Inputs principales:
# - model: El modelo PEFT preparado (de la celda 6).
# - tokenizer: El tokenizer cargado (de la celda 5).
# - args: El objeto SFTConfig con las configuraciones de entrenamiento.
# - train_dataset: El dataset de entrenamiento procesado (de la celda 8).
# - eval_dataset: El dataset de validación procesado (de la celda 8).
# - data_collator: Una función para agrupar ejemplos del dataset en batches.
print(f"  Input model type: {type(model)}")
print(f"  Input tokenizer type: {type(tokenizer)}")
#print(f"  Input args (SFTConfig): {sft_config}")
print(f"  Input train_dataset size: {len(train_ds)} ejemplos")
print(f"  Input eval_dataset size: {len(val_ds)} ejemplos")
print(f"  Input data_collator: {default_data_collator}")

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=sft_config,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    data_collator=default_data_collator,
)
print("SFTTrainer inicializado.")
# Output principal: Un objeto SFTTrainer.


# Para iniciar el entrenamiento, descomenta la línea trainer.train()
print("\n--- Listo para iniciar el entrenamiento ---")
trainer.train()
print("--- Entrenamiento finalizado ---")

# Para guardar el adaptador después del entrenamiento, descomenta estas líneas
print(f"\n--- Guardando adaptador PEFT en {CFG.OUTPUT_DIR}/{CFG.ADAPTER_NAME} ---")
trainer.model.save_pretrained(f"{CFG.OUTPUT_DIR}/{CFG.ADAPTER_NAME}")
tokenizer.save_pretrained(CFG.OUTPUT_DIR) # Guarda el tokenizer también
print("--- Adaptador y tokenizer guardados ---")

In [ ]:
# ============================================================
# 10) Inferencia de prueba
# ============================================================
import torch
from peft import PeftModel

def chat(prompt: str, sys: str = "Eres un asistente experto en Arquitectura de Software."):
    messages = [
        {"role":"system","content": sys},
        {"role":"user","content": prompt}
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # --- Agregar logs aquí ---
    print("\n--- Detalles de Inferencia ---")
    print(f"Modelo base: {CFG.BASE_MODEL}")
    print(f"Usando PEFT/LoRA: {isinstance(model, PeftModel)}")
    print(f"Dispositivo del modelo: {model.device}")
    print(f"Texto de entrada tokenizado: {text[:500]}...") # Imprime los primeros 500 caracteres
    print(f"Longitud del texto de entrada: {len(text)}")
    # --------------------------

    inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # --- Más logs sobre los inputs ---
    print(f"Inputs tensor shape: {inputs['input_ids'].shape}")
    print(f"Inputs tensor device: {inputs['input_ids'].device}")
    print(f"Attention mask shape: {inputs['attention_mask'].shape}")
    print(f"Attention mask device: {inputs['attention_mask'].device}")
    print(f"Max new tokens: {CFG.MAX_NEW_TOKENS}")
    print(f"Temperature: {CFG.TEMPERATURE}")
    print(f"Top P: {CFG.TOP_P}")
    print("-----------------------------")
    # -------------------------------

    model.eval()
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=CFG.MAX_NEW_TOKENS,
            temperature=CFG.TEMPERATURE,
            top_p=CFG.TOP_P,
            do_sample=True
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print(chat("En terminos simples, explicame de que trata arquitectura dirigida de microservicios"))

In [ ]:
# ============================================================
# 11) (Opcional) Merge del adaptador y exportación
# ============================================================
from peft import AutoPeftModelForCausalLM
import os
import torch # Import torch

# Define an offload directory (still needed if device_map is not used but model is large)
offload_directory = "/tmp/offload"
os.makedirs(offload_directory, exist_ok=True)

# Determine device
device = "cuda" if torch.cuda.is_available() else "cpu"

merged = AutoPeftModelForCausalLM.from_pretrained(
    f"{CFG.OUTPUT_DIR}/{CFG.ADAPTER_NAME}",
    # device_map="auto", # Remove auto device mapping
    # Use explicit device if needed, or rely on default
    offload_folder=offload_directory # Keep offload directory as a fallback/option
).to(device) # Explicitly move to device

merged = merged.merge_and_unload()
merged.save_pretrained(f"{CFG.OUTPUT_DIR}/merged", safe_serialization=True)
tokenizer.save_pretrained(f"{CFG.OUTPUT_DIR}/merged")


## Troubleshooting (T4 + CUDA 12.6)
- **bnb sin GPU / no lib cuda** → Repite la celda **0** y luego reinicia runtime. Verifica en **1)** que aparezca `libbitsandbytes_cuda126.so`.
- **`bfloat16` no soportado** → Ya configurado (`USE_BF16=False`, `USE_FP16=True`).
- **OOM** → Baja `MAX_SEQ_LEN` (1024→768/512), deja `BATCH=1`, mantén `GRADIENT_ACCUMULATION` alto, `gradient_checkpointing=True`.
- **labels/pad** → Función de tokenización convierte PAD→`-100`.
- **flash-attn/xformers** → Opcionales; SDPA de PyTorch es suficiente en T4.

### Resumen didáctico
- **[TRANSFORMER]**: celda **5** instancia `AutoModelForCausalLM` (Llama 3.x); **celda 6** inyecta LoRA en `q/k/v/o` y MLP.  
- **[DATA TRANSFORM]**: celda **8** aplica `apply_chat_template` → `tokenizer` (trunc/pad) → `labels` (pad = -100).
